# Recap

## Data 

Given the next function, load the data

In [0]:
from tensorflow.keras.datasets import imdb

def load_data():
    (sentences_train, y_train), (sentences_test, y_test) = imdb.load_data()
    word_to_id = imdb.get_word_index()

    word_to_id = {k:(v+3) for k,v in word_to_id.items()}
    for i, w in enumerate(['<PAD>', '<START>', '<UNK>', '<UNUSED>']):
        word_to_id[w] = i

    id_to_word = {v:k for k, v in word_to_id.items()}

    sentences_train = [[id_to_word[_] for _ in sentence] for sentence in sentences_train]
    sentences_test = [[id_to_word[_] for _ in sentence] for sentence in sentences_test]
    
    return sentences_train, y_train, sentences_test, y_test

sentences_train, y_train, sentences_test, y_test = load_data()

When using text, you have to convert your text into meaningful representation. You have two options to do that:
- option #1: learn a representation, thanks to word2vec for instance, and then input it in the model
- option #2: learn the representation directly in the model

## Option #1: Word2Vec

❓ Learn a word2vec representation, for which you can defined size of the embedding, the learning window and the minimum occurences of a word to count it. 

In [0]:
# YOUR CODE HERE

❓ Convert your sentences to their embedded representation

In [0]:
# YOUR CODE HERE

❓ Pad your input data

In [0]:
# YOUR CODE HERE

❓ Initialize a model and run it

In [0]:
# YOUR CODE HERE

# Option 2: Embedding layer

Instead of learning an embedding that is not related to the task you are working on, you can learn the vector representation of the words with an embedding layer in the model. 

❓ But first, we have to convert the words in the sentences to tokens (because word2vec does it internally). Create a dictionary whose keys are the words in the train set, and the values are unique integer identifiers.

In [0]:
# YOUR CODE HERE

❓ Convert your train and test sentences to lists of tokens

In [0]:
# YOUR CODE HERE

❓ Pad your data

In [0]:
# YOUR CODE HERE

❓ Write, in a function, a model which has:
- a masking layer not to take into account padded values
- an embedding for which you can choose the embedding size
- a LSTM or a GRU layer
- a dense layer
- an output layer

Then, compile your model

In [0]:
# YOUR CODE HERE

❓ Fit your model on the train data

In [0]:
# YOUR CODE HERE